In [1]:
with open('input05.txt') as f:
    opcodes = [int(n) for n in f.read().split(',')]

In [2]:
class Intcode():
    def __init__(self, memory, noun=None, verb=None):
        self.memory = memory.copy()
        if noun:
            self.memory[1] = noun
        if verb:
            self.memory[2] = verb
        
        self.ops = {
            # op, num_reads, num_writes
            1:  ('add', 2, 1),
            2:  ('mul', 2, 1),
            3:  ('inp', 0, 1),
            4:  ('out', 1, 0),
            5:  ('jit', 2, 0),
            6:  ('jif', 2, 0),
            7:  ('les', 2, 1),
            8:  ('equ', 2, 1),
            99: ('hlt', 0, 0)}
        
        self.ptr = 0
    
    def run(self):
        while True:
            op, args = self.getargs(self.ptr)
            if op == 'hlt':
                return
            else:
                # pass arguments to function called op
                # advance pointer
                adv = getattr(self, op)(*args)
                self.ptr += adv
    
    def getargs(self, ptr):
        instr = self.memory[ptr]
        opcode = instr % 100
        instr //= 100
        op, ninputs, noutputs = self.ops[opcode]
        args = []
        for i in range(ninputs):
            parmode = instr % 10
            instr //= 10
            par = self.memory[ptr+1+i]
            args.append(par if parmode == 1 else self.memory[par])
        for i in range(noutputs):
            par = self.memory[ptr+ninputs+1+i]
            args.append(par)
        return op, args
    
    def add(self, p1, p2, p3):
        self.memory[p3] = p1 + p2
        return 4
    
    def mul(self, p1, p2, p3):
        self.memory[p3] = p1 * p2
        return 4
    
    def inp(self, p1):
        self.memory[p1] = int(input())
        return 2
    
    def out(self, p1):
        print(p1)
        return 2

    def jit(self, p1, p2):
        if p1:
            self.ptr = p2
            return 0
        else:
            return 3
    
    def jif(self, p1, p2):
        if not p1:
            self.ptr = p2
            return 0
        else:
            return 3
    
    def les(self, p1, p2, p3):
        self.memory[p3] = int(p1 < p2)
        return 4
    
    def equ(self, p1, p2, p3):
        self.memory[p3] = int(p1 == p2)
        return 4

In [3]:
intcode = Intcode(opcodes)
intcode.run()

11189491
